In [1]:
### imports

# external modules
import os
import sys
import time
import numpy as np
import matplotlib.pyplot as plt
import importlib
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow import keras
from keras import backend as K
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
from tensorflow.keras.layers import Input, Dense, Concatenate
from tensorflow.keras.models import Model, Sequential, load_model
import importlib
from sklearn.preprocessing import StandardScaler

# local modules
sys.path.append('../utils')
import csv_utils as csvu
import json_utils as jsonu
import dataframe_utils as dfu
import hist_utils as hu
import autoencoder_utils as aeu
import plot_utils as pu
import generate_data_utils as gdu
import refruns_utils as rru
importlib.reload(csvu)
importlib.reload(jsonu)
importlib.reload(dfu)
importlib.reload(hu)
importlib.reload(aeu)
importlib.reload(pu)
importlib.reload(gdu)
importlib.reload(rru)
sys.path.append('../src')
sys.path.append('../src/classifiers')
sys.path.append('../src/cloudfitters')
import HistStruct
importlib.reload(HistStruct)
import SubHistStruct
importlib.reload(SubHistStruct)
import FlexiStruct
importlib.reload(FlexiStruct)
import DataLoader
importlib.reload(DataLoader)
import AutoEncoder
importlib.reload(AutoEncoder)
import SeminormalFitter
import GaussianKdeFitter
import HyperRectangleFitter
importlib.reload(SeminormalFitter)
importlib.reload(GaussianKdeFitter)
importlib.reload(HyperRectangleFitter)

2022-07-28 15:58:00.632065: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /cvmfs/sft.cern.ch/lcg/releases/MCGenerators/thepeg/2.2.3-88592/x86_64-centos7-gcc11-opt/lib/ThePEG:/cvmfs/sft.cern.ch/lcg/releases/MCGenerators/herwig++/7.2.3-35f7a/x86_64-centos7-gcc11-opt/lib/Herwig:/cvmfs/sft.cern.ch/lcg/views/LCG_102swan/x86_64-centos7-gcc11-opt/lib/python3.9/site-packages/jaxlib/mlir/_mlir_libs:/cvmfs/sft.cern.ch/lcg/views/LCG_102swan/x86_64-centos7-gcc11-opt/lib/python3.9/site-packages/torch/lib:/cvmfs/sft.cern.ch/lcg/views/LCG_102swan/x86_64-centos7-gcc11-opt/lib/python3.9/site-packages/onnxruntime/capi/:/cvmfs/sft.cern.ch/lcg/views/LCG_102swan/x86_64-centos7-gcc11-opt/lib/python3.9/site-packages/tensorflow:/cvmfs/sft.cern.ch/lcg/views/LCG_102swan/x86_64-centos7-gcc11-opt/lib/python3.9/site-packages/tensorflow/contrib

<module 'HyperRectangleFitter' from '/eos/home-i01/k/khowey/SWAN_projects/ML4DQMDC-PixelAE/KH-AutoencoderTest/../src/cloudfitters/HyperRectangleFitter.py'>

In [2]:
### Speed Controls and Run Mode

# Disables all plots for large datasets where speed is more important
createPlots = False

In [3]:
### Evaluation Parameters

# Select the bias towards recall against precision, treated as a factor (so < 1 biases towards precision, 1 is equal importance, and > 1 biases towards recall)
wpBiasFactor = 20
fmBiasFactor = 2

In [4]:
### Defining bad runs
badruns = {'2017B':
                [
                    297048,
                    297282,
                    297283,
                    297284,
                    297287,
                    297288,
                    297289,
                    299316,
                    299317,
                    299318,
                    299324,
                    299326,
                    301086,
                    301086,
                    303948,
                    297047, #close but, true bad for all 8
                    297169, #true bad for all 8
                    297211, #Reconstructs well
                    299325, #Reconstructs well
                    297664, #true bad for all 8
                    299317, #true bad for all 8
                    297169, #true bad for all 8
                    297502
                ],
             '2017C':[
                  300781, # bad for tracking (pixels were excluded.
                  300079, # is bad for strips and then also for tracking
                  302029, # Poor detector elements for strips - Should be mildly anomalous, but technically good 
                  300576, # Poor detector elements for strips - Should be mildly anomalous, but technically good
                  300574, # Poor detector elements for strips - Should be mildly anomalous, but technically good
                  300282, # Poor detector elements for strips - Should be mildly anomalous, but technically good
                  301912, # Half bad for pixels (lost HV or readout card)  
                  301086, # Half bad for pixels (lost HV or readout card)  
                  300283, # Half bad for pixels (lost HV or readout card) 
                  300282, # Half bad for pixels (lost HV or readout card) 
                  300281, # Half bad for pixels (lost HV or readout card) 
                  300239, # Half bad for pixels (lost HV or readout card)
                  301394, # Marginal for pixels
                  301183, # Marginal for pixels
                  300398, # Marginal for pixels
                  300389, # Marginal for pixels
                  300365  # Marginal for pixels
             ],
             '2017E':[
                 304740, # Bad for pixels and tracking - holes in PXLayer 1
                 304776, # Bad for pixels and tracking - holes in PXLayer 1
                 304506, # Portcard problem for pixels
                 304507, # Portcard problem for pixels 
                 303989, # Bad for pixels, power supply died
                 303824  # Partly bad for strips due to a test
             ],
             '2017F':[
                 306422, # Partly bad for strips - 2 data readouts failed 
                 306423, # Partly bad for strips - 2 data readouts failed
                 306425, # Partly bad for strips - 2 data readouts failed
                 305440, # Partly bad for strips - 1 data readout failed
                 305441, # Partly bad for strips - 1 data readout failed
                 305249, # Bad for pixels - half of disk failed 
                 305250, # Bad for pixels - half of disk failed
                 305064, # Marginal for pixels - some readout failed
             ],
            '2018': # needs to be re-checked, not guaranteed to be fully correct or representative.
                [
                317479,
                317480,
                317481,
                317482,
                319847
                ]}


In [16]:
### Select a reference run and get data
rundict = jsonu.loadjson('../jsons/CertHelperRefRuns.json')

# Select run numbers to get the references for
#runNums = [299325, 303824, 306422]
runNums = [297047, 299325]
refRuns = []
eras = []
years = []
dataDict = {}
badrunsls = {}
trainrunsls = {}
goodrunsls = {}
for runNum in runNums:
    runls = {}
    for run in rundict:
        if run['run_number'] == runNum:
            runls.update(run)
    if runls == {}:
        raise Exception('Run not found - ' + str(runNum))
    
    year = runls['dataset'][11:15]
    if year not in years: years.append(year)
    era = runls['dataset'][15]
    if era not in eras: eras.append(era)
    ref_run = runls['reference_run_number']
    
    # Don't need duplicates
    if ref_run in refRuns:
        continue
    refRuns.append(ref_run)
    
    # Get the runs associated with found reference
    outputRuns = {}
    outputBad = {}
    for run in rundict:
        tempRef = run['reference_run_number']
        if tempRef == ref_run:
            runls = {}
            runls[str(run['run_number'])] = [[-1]]
            if run['run_number'] in badruns[year+era]:
                print('Found bad run :' + str(run))
                outputBad.update(runls)
            else:
                outputRuns.update(runls)
    
    # Perform structuring for compatibility with autoencoders
    dataDict[year + era] = outputRuns
    badrunsls[year + era] = outputBad
    trainrunsls[year + era] = {}
    goodrunsls[year + era] = {}
    
    # Select training and testing set
    for i,run in enumerate(dataDict[year + era]):
        if i > 5 and i < 11:
            goodrunsls[year + era][str(run)] = [[-1]]
        else:
            trainrunsls[year + era][str(run)] = [[-1]]

if len(years) != 1: raise Exception('Year of length 0 or >1 unimplemented!')

Found bad run :{'run_number': 297211, 'run_reconstruction_type': 'rerecoul', 'reference_run_number': 297178, 'reference_run_reconstruction_type': 'rerecoul', 'dataset': '/ReReco/Run2017B_UL2019/DQM'}
Found bad run :{'run_number': 297169, 'run_reconstruction_type': 'rerecoul', 'reference_run_number': 297178, 'reference_run_reconstruction_type': 'rerecoul', 'dataset': '/ReReco/Run2017B_UL2019/DQM'}
Found bad run :{'run_number': 297664, 'run_reconstruction_type': 'rerecoul', 'reference_run_number': 297178, 'reference_run_reconstruction_type': 'rerecoul', 'dataset': '/ReReco/Run2017B_UL2019/DQM'}
Found bad run :{'run_number': 299325, 'run_reconstruction_type': 'rerecoul', 'reference_run_number': 297178, 'reference_run_reconstruction_type': 'rerecoul', 'dataset': '/ReReco/Run2017B_UL2019/DQM'}
Found bad run :{'run_number': 299317, 'run_reconstruction_type': 'rerecoul', 'reference_run_number': 297178, 'reference_run_reconstruction_type': 'rerecoul', 'dataset': '/ReReco/Run2017B_UL2019/DQM'}


In [17]:
### Data Controls and Selection - 1D Autoncoder

# The directory data is located in
datadir = {}
for era in eras:
    datadir[year + era] = '../data/' + year + era + '/'

# Create a list of histograms to include
# Pair histograms to be combined on the same line
histnames = [['chargeInner_PXLayer_1', 'chargeInner_PXLayer_2', 'chargeInner_PXLayer_3', 'chargeInner_PXLayer_4'], ['charge_PXDisk_+1', 'charge_PXDisk_+2', 'charge_PXDisk_+3'], ['charge_PXDisk_-1', 'charge_PXDisk_-2', 'charge_PXDisk_-3'], ['Summary_ClusterStoNCorr__OnTrack__TIB__layer__1', 'Summary_ClusterStoNCorr__OnTrack__TIB__layer__2', 'Summary_ClusterStoNCorr__OnTrack__TIB__layer__3', 'Summary_ClusterStoNCorr__OnTrack__TIB__layer__4'], ['Summary_ClusterStoNCorr__OnTrack__TOB__layer__1', 'Summary_ClusterStoNCorr__OnTrack__TOB__layer__2', 'Summary_ClusterStoNCorr__OnTrack__TOB__layer__3', 'Summary_ClusterStoNCorr__OnTrack__TOB__layer__4', 'Summary_ClusterStoNCorr__OnTrack__TOB__layer__5', 'Summary_ClusterStoNCorr__OnTrack__TOB__layer__6'], ['Summary_ClusterStoNCorr__OnTrack__TID__PLUS__wheel__1', 'Summary_ClusterStoNCorr__OnTrack__TID__PLUS__wheel__2', 'Summary_ClusterStoNCorr__OnTrack__TID__PLUS__wheel__3'], ['Summary_ClusterStoNCorr__OnTrack__TID__MINUS__wheel__1', 'Summary_ClusterStoNCorr__OnTrack__TID__MINUS__wheel__2', 'Summary_ClusterStoNCorr__OnTrack__TID__MINUS__wheel__3'], ['Summary_ClusterStoNCorr__OnTrack__TEC__PLUS__wheel__1', 'Summary_ClusterStoNCorr__OnTrack__TEC__PLUS__wheel__2', 'Summary_ClusterStoNCorr__OnTrack__TEC__PLUS__wheel__3', 'Summary_ClusterStoNCorr__OnTrack__TEC__PLUS__wheel__4', 'Summary_ClusterStoNCorr__OnTrack__TEC__PLUS__wheel__5', 'Summary_ClusterStoNCorr__OnTrack__TEC__PLUS__wheel__6', 'Summary_ClusterStoNCorr__OnTrack__TEC__PLUS__wheel__7', 'Summary_ClusterStoNCorr__OnTrack__TEC__PLUS__wheel__8', 'Summary_ClusterStoNCorr__OnTrack__TEC__PLUS__wheel__9'], ['Summary_ClusterStoNCorr__OnTrack__TEC__MINUS__wheel__1', 'Summary_ClusterStoNCorr__OnTrack__TEC__MINUS__wheel__2', 'Summary_ClusterStoNCorr__OnTrack__TEC__MINUS__wheel__3', 'Summary_ClusterStoNCorr__OnTrack__TEC__MINUS__wheel__4', 'Summary_ClusterStoNCorr__OnTrack__TEC__MINUS__wheel__5', 'Summary_ClusterStoNCorr__OnTrack__TEC__MINUS__wheel__6', 'Summary_ClusterStoNCorr__OnTrack__TEC__MINUS__wheel__7', 'Summary_ClusterStoNCorr__OnTrack__TEC__MINUS__wheel__8', 'Summary_ClusterStoNCorr__OnTrack__TEC__MINUS__wheel__9'], ['NumberOfRecHitsPerTrack_lumiFlag_GenTk']]

# Read new data or use previously saved data & should data be saved
readnew = True
save = False

In [18]:
### Define Run Properties - Combined Autoencoder
# in this cell all major run properties are going to be set,

# Set whether to train globally or locally or in a development/testing mode
training_mode = 'development'

In [19]:
### Model Controls and Selection - 1D Autoencoder

plotNames = 'Test'
name = plotNames+'plots'

# Choose whether to train a new model or load one
trainnew = True
savemodel = True
modelname = plotNames

In [20]:
### Define Training Mode Parameters - Combined Autoencoder
if training_mode == 'global':
    runsls_training = None # use none to not add a mask for training (can e.g. use DCS-bit on mask)
    runsls_good = None # use none to not add a mask for good runs (can e.g. use averages of training set)
    runsls_bad = badrunsls[year] # predefined bad runs
    print('selected runs/lumisections for training: all')
    
elif training_mode == 'local':
    # train locally on a small set of runs
    # - either on n runs preceding a chosen application run,
    # - or on the run associated as reference to the chosen application run.
    # - this only works for a single era
    
    available_runs = dfu.get_runs( dfu.select_dcson( csvu.read_csv('../data/DF'+year+era+'_'+histnames[0][0]+'.csv') ) )
    # Cherry picked really bad run
    run_application = 299316
    #run_application = 299317
    run_application_index = available_runs.index(run_application)
    # select training set
    usereference = False
    if usereference:
        run_reference = rru.get_reference_run( run_application, jsonfile='../utils/json_allRunsRefRuns.json' )
        if run_reference<0:
            raise Exception('no valid reference run has been defined for run {}'.format(run_application))
        runsls_training = jsonu.tuplelist_to_jsondict([(run_reference,[-1])])
    else:
        ntraining = 5
        offset = 0 # normal case: offset = 0 (just use 5 previous runs)
        
        # Selects the 5 previous runs for training
        runsls_training = jsonu.tuplelist_to_jsondict([(el,[-1]) for el in available_runs[run_application_index-ntraining-offset:run_application_index-offset]])
    #runsls_bad = badrunsls[year]
    #runsls_good = jsonu.tuplelist_to_jsondict([(run_application,[-1])])
    runsls_bad = jsonu.tuplelist_to_jsondict([(run_application,[-1])])
    runsls_good = runsls_training
    print('selected runs/lumisections for training: ')
    print(runsls_training)
    print('selected runs/lumisections as good test set:')
    print(runsls_good)
    print('selected runs/lumisections as bad test set:')
    print(runsls_bad)
        
elif training_mode == 'development':
    # train on a user-defined subset of runs
    
   # Select runs to be used in training from the user-defined list
    runsls_training = {}
    runsls_bad = {}
    runsls_good = {}
    for era in eras:
        runsls_training.update(trainrunsls[year + era])
        # Select bad runs to test on in the user-defined list
        runsls_bad.update(badrunsls[year + era])
        # Select good runs to test on in the user-defined list
        runsls_good.update(goodrunsls[year + era])
    
    print('selected runs/lumisections for training: ')
    print(runsls_training)
    print('selected runs/lumisections as good test set:')
    print(runsls_good)
    print('selected runs/lumisections as bad test set:')
    print(runsls_bad)

selected runs/lumisections for training: 
{'297050': [[-1]], '297057': [[-1]], '297099': [[-1]], '297101': [[-1]], '297113': [[-1]], '297114': [[-1]], '297426': [[-1]], '297225': [[-1]], '297431': [[-1]], '297434': [[-1]], '297468': [[-1]], '297474': [[-1]], '297483': [[-1]], '297296': [[-1]], '297486': [[-1]], '297503': [[-1]], '297557': [[-1]], '297598': [[-1]], '297604': [[-1]], '297620': [[-1]], '297659': [[-1]], '297670': [[-1]], '297674': [[-1]], '298997': [[-1]], '299000': [[-1]], '299096': [[-1]], '297678': [[-1]], '297722': [[-1]], '299061': [[-1]], '299065': [[-1]], '299067': [[-1]], '299149': [[-1]], '299178': [[-1]], '299184': [[-1]], '299185': [[-1]], '299327': [[-1]], '299329': [[-1]], '299480': [[-1]], '297175': [[-1]]}
selected runs/lumisections as good test set:
{'297177': [[-1]], '297179': [[-1]], '297215': [[-1]], '297218': [[-1]], '297411': [[-1]]}
selected runs/lumisections as bad test set:
{'297211': [[-1]], '297169': [[-1]], '297664': [[-1]], '299325': [[-1]], '2

In [21]:
### Data Import

# Create a new HistStruct from the data
if readnew:
    # Initializations
    dloader = DataLoader.DataLoader()
    histstruct = FlexiStruct.FlexiStruct()
    histstruct.reset_histlist(histnames)
    
    # Unpack histnames and add every histogram individually
    for era in eras:
        print('Preparing data for era ' + era)
        for histnamegroup in histnames:
            for histname in histnamegroup:
                print('Adding {}...'.format(histname))
                
                # Bring the histograms into memory from storage for later use
                filename = datadir[year+era] + 'DF' + year + era + '_' + histname + '.csv'
                df = dloader.get_dataframe_from_file( filename )
                
                # In case of local training, we can remove most of the histograms
                if( runsls_training is not None and runsls_good is not None and runsls_bad is not None ):
                    runsls_total = {k: v for d in (runsls_training, runsls_good, runsls_bad) for k, v in d.items()}
                    df = dfu.select_runsls( df, runsls_total )    
                
                df = dfu.rm_duplicates(df)
                # Store the data in the histstruct object managing this whole thing
                histstruct.add_dataframe( df, rebinningfactor = 1, standardbincount = 102 )
        print('Found {} histograms'.format(len(histstruct.runnbs)))
        print()

# Load a previously saved HistStruct
else:
    # Load histstruct from storage
    histstruct = SubHistStruct.SubHistStruct.load( 'histstruct_global_20220201.zip', verbose=False )
    nbadruns = len([name for name in list(histstruct.masks.keys()) if ('bad' in name and name!='bad')])
    
    print('loaded a histstruct with the following properties:')
    print(histstruct)
    # Count of bad runs, presumably for later use
    nbadruns = len([name for name in list(histstruct.masks.keys()) if 'bad' in name])
    
print('Created a histstruct with the following properties:')
print('- number of histogram types: {}'.format(len(histstruct.histnames)))
print('- number of lumisections: {}'.format(len(histstruct.lsnbs)))

Preparing data for era B
Adding chargeInner_PXLayer_1...
Adding chargeInner_PXLayer_2...
Adding chargeInner_PXLayer_3...
Adding chargeInner_PXLayer_4...
Adding charge_PXDisk_+1...
Adding charge_PXDisk_+2...
Adding charge_PXDisk_+3...
Adding charge_PXDisk_-1...
Adding charge_PXDisk_-2...
Adding charge_PXDisk_-3...
Adding Summary_ClusterStoNCorr__OnTrack__TIB__layer__1...
Adding Summary_ClusterStoNCorr__OnTrack__TIB__layer__2...
Adding Summary_ClusterStoNCorr__OnTrack__TIB__layer__3...
Adding Summary_ClusterStoNCorr__OnTrack__TIB__layer__4...
Adding Summary_ClusterStoNCorr__OnTrack__TOB__layer__1...
Adding Summary_ClusterStoNCorr__OnTrack__TOB__layer__2...
Adding Summary_ClusterStoNCorr__OnTrack__TOB__layer__3...
Adding Summary_ClusterStoNCorr__OnTrack__TOB__layer__4...
Adding Summary_ClusterStoNCorr__OnTrack__TOB__layer__5...
Adding Summary_ClusterStoNCorr__OnTrack__TOB__layer__6...
Adding Summary_ClusterStoNCorr__OnTrack__TID__PLUS__wheel__1...
Adding Summary_ClusterStoNCorr__OnTrack__

In [22]:
### Add Masks to Data

if readnew:
    histstruct.add_dcsonjson_mask( 'dcson' )
    histstruct.add_goldenjson_mask('golden' )
    histstruct.add_highstat_mask( 'highstat', entries_to_bins_ratio=0 )
    histstruct.add_stat_mask( 'lowstat', max_entries_to_bins_ratio=0 )
    if runsls_training is not None: histstruct.add_json_mask( 'training', runsls_training )
    if runsls_good is not None: histstruct.add_json_mask( 'good', runsls_good )
        
    # Distinguishing bad runs
    nbadruns = 0
    if runsls_bad is not None:
        print(runsls_bad)
        histstruct.add_json_mask( 'bad', runsls_bad )
        
        # Special case for bad runs: add a mask per run (different bad runs have different characteristics)
        nbadruns = len(runsls_bad.keys())
        for i,badrun in enumerate(runsls_bad.keys()):
            histstruct.add_json_mask( 'bad{}'.format(i), {badrun:runsls_bad[badrun]} )
            
    if save:
        histstruct.save('test.pk1')
print('Assigned masks: {}'.format(list(histstruct.masks.keys())))

{'297211': [[-1]], '297169': [[-1]], '297664': [[-1]], '299325': [[-1]], '299317': [[-1]], '297047': [[-1]]}
Assigned masks: ['dcson', 'golden', 'highstat', 'lowstat', 'training', 'good', 'bad', 'bad0', 'bad1', 'bad2', 'bad3', 'bad4', 'bad5']


In [23]:
### Plotting the input data for analysis

if((training_mode=='local' or training_mode == 'development') and createPlots):

    # training and application runs
    histstruct.plot_histograms( masknames=[['dcson','highstat','training'],['dcson','highstat','good']],
                                labellist = ['training','testing'],
                                colorlist = ['blue','green']
                              )
    
    # application run and bad test runs
    histstruct.plot_histograms( masknames=[['dcson','highstat','good'],['dcson','highstat','bad0']],
                                labellist = ['good','bad'],
                                colorlist = ['green','red']
                              )
    
elif( training_mode=='global' and createPlots):
    
    # bad test runs
    for i in [0,1,2,3,4,5,6]:
        histstruct.plot_histograms( masknames=[['dcson','highstat','good'],['dcson','highstat','bad{}'.format(i)]],
                                labellist = ['typical good histograms','bad'],
                                colorlist = ['blue','red'],
                                transparencylist = [0.01,1.]
                                    )

In [24]:
def define_concatamash_autoencoder(histstruct):
    
    histslist = []
    vallist = []
    autoencoders = []
    if trainnew:
        for i,histnamegroup in enumerate(histnames):
            
            train_normhist = np.array([hu.normalize(histstruct.get_histograms(
                histname = hname, masknames = ['dcson','highstat', 'training']), 
                                                 norm="l1", axis=1) 
                                       for hname in histnamegroup]).transpose((1,0,2))
            X_train, X_val = train_test_split(train_normhist, test_size=0.4, random_state=42)
            
            # Half the total bin count
            arch = 51 * len(histnamegroup)
            
            ## Model parameters
            input_dim = X_train.shape[2] #num of predictor variables
            Input_layers=[Input(shape=input_dim) for i in range((X_train.shape[1]))]
            
            # Defining layers
            conc_layer = Concatenate()(Input_layers)
            encoder = Dense(arch * 2, activation="tanh")(conc_layer)
            encoder = Dense(arch/2, activation='relu')(encoder)
            encoder = Dense(arch/8, activation='relu')(encoder)
            encoder = Dense(arch/16, activation='relu')(encoder)
            decoder = Dense(arch/8, activation="relu")(encoder)
            decoder = Dense(arch/2, activation='relu')(encoder)
            decoder = Dense(arch * 2, activation="tanh")(decoder)
            
            Output_layers=[Dense(input_dim, activation="tanh")(decoder) for i in range(X_train.shape[1])]

            autoencoder = Model(inputs=Input_layers, outputs=Output_layers)
            autoencoders.append(autoencoder)
            
            histslist.append(X_train)
            vallist.append(X_val)
     
    # Return the histograms stored 2-Dimensionally and the autoencoders corresponding
    return(histslist, vallist, autoencoders, train_normhist)

In [25]:
(histslist, vallist, autoencoders, train_normhist) = define_concatamash_autoencoder(histstruct)

2022-07-28 16:09:07.222258: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /cvmfs/sft.cern.ch/lcg/releases/MCGenerators/thepeg/2.2.3-88592/x86_64-centos7-gcc11-opt/lib/ThePEG:/cvmfs/sft.cern.ch/lcg/releases/MCGenerators/herwig++/7.2.3-35f7a/x86_64-centos7-gcc11-opt/lib/Herwig:/cvmfs/sft.cern.ch/lcg/views/LCG_102swan/x86_64-centos7-gcc11-opt/lib/python3.9/site-packages/jaxlib/mlir/_mlir_libs:/cvmfs/sft.cern.ch/lcg/views/LCG_102swan/x86_64-centos7-gcc11-opt/lib/python3.9/site-packages/torch/lib:/cvmfs/sft.cern.ch/lcg/views/LCG_102swan/x86_64-centos7-gcc11-opt/lib/python3.9/site-packages/onnxruntime/capi/:/cvmfs/sft.cern.ch/lcg/views/LCG_102swan/x86_64-centos7-gcc11-opt/lib/python3.9/site-packages/tensorflow:/cvmfs/sft.cern.ch/lcg/views/LCG_102swan/x86_64-centos7-gcc11-opt/lib/python3.9/site-packages/tensorflow/contrib/tensor_fo

In [15]:
### Trains a combined autoencoder for every merge set
def train_concatamash_autoencoder(histstruct, histslist, vallist, autoencoders):
    
    autoencodersTrain = []
    for i in range(len(histslist)):
        
        print('Now training model {}/'.format(i + 1) + str(len(histslist)))
        
        # Set variables to temporary values for better transparency
        X_train = histslist[i]
        X_val = vallist[i]
        autoencoder = autoencoders[i]
        
        
        ## Model parameters
        nb_epoch = 500
        batch_size = 10000
        
        #checkpoint_filepath = 'checkpoint'
        #model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
        #   filepath=checkpoint_filepath,
        #   save_weights_only=False,
        #   verbose=1,
        #   save_best_only=True,
        #   monitor='val_loss',
        #   mode='min')
        
        # Tell the model when to stop
        earlystop = EarlyStopping(monitor='val_loss',
            min_delta=1e-7,
            patience=20,
            verbose=1,
            mode='auto',
            baseline=None,
            restore_best_weights=True,
        )
        lr =0.001
        opt = keras.optimizers.Adam(learning_rate=lr)
        
        autoencoder.compile(loss='mse',
                            optimizer=opt)
        
        ## Train autoencoder
        train = autoencoder.fit(x=[X_train[:,i] for i in range(X_train.shape[1])],
                                y=[X_train[:,i] for i in range(X_train.shape[1])],
                                epochs=nb_epoch,
                                batch_size=batch_size,
                                shuffle=True,
                                validation_data=([X_val[:,i] for i in range(X_val.shape[1])], [X_val[:,i] for i in range(X_val.shape[1])]),
                                verbose=1,
                                callbacks= [earlystop],    
                                )
        tf.keras.utils.plot_model(
                    autoencoder,
                    to_file="models/model1D{}.png".format(i),
                    show_shapes=True,
                    show_dtype=False,
                    show_layer_names=False,
                    rankdir="TB")
        
        # Save classifier for evaluation
        classifier = AutoEncoder.AutoEncoder(model=autoencoder)
        histstruct.add_classifier(histnames[i][0], classifier) 
        autoencodersTrain.append(classifier)
        K.clear_session()
        del(autoencoder, classifier)
    return autoencodersTrain

In [16]:
def load_concatamash_autoencoder():
    for i in range(len(histnames)):
        autoencoder = tf.keras.models.load_model('../SavedModels/Permutations/Job1/AE{}'.format(i))
        classifier = AutoEncoder.AutoEncoder(model=autoencoder)
        histstruct.add_classifier(histnames[i][0], classifier)

In [17]:
start = time.perf_counter()
if trainnew: train_concatamash_autoencoder(histstruct, histslist, vallist, autoencoders)
else: load_concatamash_autoencoder()
stop = time.perf_counter()
print(stop - start)

Now training model 1/10
Epoch 1/500
2/2 [==============================] - 2s 624ms/step - loss: 0.0015 - dense_7_loss: 3.1059e-04 - dense_8_loss: 3.9352e-04 - dense_9_loss: 3.7105e-04 - dense_10_loss: 3.8147e-04 - val_loss: 9.9931e-04 - val_dense_7_loss: 2.2246e-04 - val_dense_8_loss: 2.7476e-04 - val_dense_9_loss: 2.6188e-04 - val_dense_10_loss: 2.4020e-04
Epoch 2/500
2/2 [==============================] - 0s 97ms/step - loss: 9.7171e-04 - dense_7_loss: 2.1650e-04 - dense_8_loss: 2.6777e-04 - dense_9_loss: 2.5412e-04 - dense_10_loss: 2.3332e-04 - val_loss: 6.1928e-04 - val_dense_7_loss: 1.4040e-04 - val_dense_8_loss: 1.7708e-04 - val_dense_9_loss: 1.5572e-04 - val_dense_10_loss: 1.4607e-04
Epoch 3/500
2/2 [==============================] - 0s 104ms/step - loss: 5.9689e-04 - dense_7_loss: 1.3532e-04 - dense_8_loss: 1.7117e-04 - dense_9_loss: 1.4986e-04 - dense_10_loss: 1.4055e-04 - val_loss: 3.1778e-04 - val_dense_7_loss: 7.3569e-05 - val_dense_8_loss: 9.6728e-05 - val_dense_9_loss: 7

Epoch 25/500
2/2 [==============================] - 0s 98ms/step - loss: 3.5208e-05 - dense_7_loss: 8.5763e-06 - dense_8_loss: 1.2350e-05 - dense_9_loss: 9.4276e-06 - dense_10_loss: 4.8535e-06 - val_loss: 3.3765e-05 - val_dense_7_loss: 8.1545e-06 - val_dense_8_loss: 1.1962e-05 - val_dense_9_loss: 8.9716e-06 - val_dense_10_loss: 4.6765e-06
Epoch 26/500
2/2 [==============================] - 0s 96ms/step - loss: 3.4247e-05 - dense_7_loss: 8.3573e-06 - dense_8_loss: 1.2098e-05 - dense_9_loss: 9.1280e-06 - dense_10_loss: 4.6640e-06 - val_loss: 3.2988e-05 - val_dense_7_loss: 7.9619e-06 - val_dense_8_loss: 1.1733e-05 - val_dense_9_loss: 8.7494e-06 - val_dense_10_loss: 4.5436e-06
Epoch 27/500
2/2 [==============================] - 0s 119ms/step - loss: 3.3475e-05 - dense_7_loss: 8.1606e-06 - dense_8_loss: 1.1868e-05 - dense_9_loss: 8.9078e-06 - dense_10_loss: 4.5379e-06 - val_loss: 3.2419e-05 - val_dense_7_loss: 7.8475e-06 - val_dense_8_loss: 1.1520e-05 - val_dense_9_loss: 8.5908e-06 - val_de

Epoch 49/500
2/2 [==============================] - 0s 61ms/step - loss: 1.1637e-05 - dense_7_loss: 4.0516e-06 - dense_8_loss: 3.2316e-06 - dense_9_loss: 2.4256e-06 - dense_10_loss: 1.9281e-06 - val_loss: 1.0548e-05 - val_dense_7_loss: 3.7155e-06 - val_dense_8_loss: 2.8673e-06 - val_dense_9_loss: 2.1372e-06 - val_dense_10_loss: 1.8276e-06
Epoch 50/500
2/2 [==============================] - 0s 51ms/step - loss: 1.0565e-05 - dense_7_loss: 3.7776e-06 - dense_8_loss: 2.8423e-06 - dense_9_loss: 2.1661e-06 - dense_10_loss: 1.7794e-06 - val_loss: 9.5786e-06 - val_dense_7_loss: 3.4552e-06 - val_dense_8_loss: 2.5255e-06 - val_dense_9_loss: 1.9070e-06 - val_dense_10_loss: 1.6909e-06
Epoch 51/500
2/2 [==============================] - 0s 96ms/step - loss: 9.5832e-06 - dense_7_loss: 3.5117e-06 - dense_8_loss: 2.4972e-06 - dense_9_loss: 1.9335e-06 - dense_10_loss: 1.6408e-06 - val_loss: 8.7356e-06 - val_dense_7_loss: 3.2105e-06 - val_dense_8_loss: 2.2416e-06 - val_dense_9_loss: 1.7123e-06 - val_den

2/2 [==============================] - 0s 102ms/step - loss: 4.5670e-06 - dense_7_loss: 1.9998e-06 - dense_8_loss: 8.9574e-07 - dense_9_loss: 8.6204e-07 - dense_10_loss: 8.0936e-07 - val_loss: 4.5072e-06 - val_dense_7_loss: 1.9504e-06 - val_dense_8_loss: 8.8623e-07 - val_dense_9_loss: 8.1767e-07 - val_dense_10_loss: 8.5288e-07
Epoch 74/500
2/2 [==============================] - 0s 118ms/step - loss: 4.4812e-06 - dense_7_loss: 1.9867e-06 - dense_8_loss: 8.6055e-07 - dense_9_loss: 8.3602e-07 - dense_10_loss: 7.9791e-07 - val_loss: 4.4492e-06 - val_dense_7_loss: 1.9546e-06 - val_dense_8_loss: 8.5911e-07 - val_dense_9_loss: 7.9493e-07 - val_dense_10_loss: 8.4049e-07
Epoch 75/500
2/2 [==============================] - 0s 117ms/step - loss: 4.4342e-06 - dense_7_loss: 1.9845e-06 - dense_8_loss: 8.4137e-07 - dense_9_loss: 8.1885e-07 - dense_10_loss: 7.8947e-07 - val_loss: 4.5228e-06 - val_dense_7_loss: 1.9797e-06 - val_dense_8_loss: 8.8665e-07 - val_dense_9_loss: 8.1152e-07 - val_dense_10_loss

2/2 [==============================] - 0s 114ms/step - loss: 4.3747e-06 - dense_7_loss: 1.8917e-06 - dense_8_loss: 8.5301e-07 - dense_9_loss: 8.3296e-07 - dense_10_loss: 7.9698e-07 - val_loss: 4.3267e-06 - val_dense_7_loss: 1.8489e-06 - val_dense_8_loss: 8.4637e-07 - val_dense_9_loss: 7.9254e-07 - val_dense_10_loss: 8.3893e-07
Epoch 98/500
2/2 [==============================] - 0s 125ms/step - loss: 4.2991e-06 - dense_7_loss: 1.8687e-06 - dense_8_loss: 8.2738e-07 - dense_9_loss: 8.1367e-07 - dense_10_loss: 7.8943e-07 - val_loss: 4.2755e-06 - val_dense_7_loss: 1.8294e-06 - val_dense_8_loss: 8.3063e-07 - val_dense_9_loss: 7.8093e-07 - val_dense_10_loss: 8.3458e-07
Epoch 99/500
2/2 [==============================] - 0s 107ms/step - loss: 4.2497e-06 - dense_7_loss: 1.8542e-06 - dense_8_loss: 8.1006e-07 - dense_9_loss: 8.0124e-07 - dense_10_loss: 7.8415e-07 - val_loss: 4.2505e-06 - val_dense_7_loss: 1.8180e-06 - val_dense_8_loss: 8.2381e-07 - val_dense_9_loss: 7.7564e-07 - val_dense_10_loss

...

2/2 [==============================] - 1s 295ms/step - loss: 6.0100e-06 - dense_98_loss: 4.3552e-07 - dense_99_loss: 5.9960e-07 - dense_100_loss: 5.7800e-07 - dense_101_loss: 5.6107e-07 - dense_102_loss: 6.1689e-07 - dense_103_loss: 6.1469e-07 - dense_104_loss: 7.3251e-07 - dense_105_loss: 8.2666e-07 - dense_106_loss: 1.0450e-06 - val_loss: 6.2231e-06 - val_dense_98_loss: 4.7757e-07 - val_dense_99_loss: 4.9708e-07 - val_dense_100_loss: 4.6783e-07 - val_dense_101_loss: 5.8275e-07 - val_dense_102_loss: 5.1108e-07 - val_dense_103_loss: 7.3205e-07 - val_dense_104_loss: 8.1751e-07 - val_dense_105_loss: 9.0488e-07 - val_dense_106_loss: 1.2324e-06
Epoch 245/500
2/2 [==============================] - 1s 284ms/step - loss: 5.9990e-06 - dense_98_loss: 4.3409e-07 - dense_99_loss: 5.9822e-07 - dense_100_loss: 5.7647e-07 - dense_101_loss: 5.6005e-07 - dense_102_loss: 6.1594e-07 - dense_103_loss: 6.1323e-07 - dense_104_loss: 7.3136e-07 - dense_105_loss: 8.2536e-07 - dense_106_loss: 1.0443e-06 - val_

2/2 [==============================] - 1s 289ms/step - loss: 5.7636e-06 - dense_98_loss: 4.1041e-07 - dense_99_loss: 5.7438e-07 - dense_100_loss: 5.4831e-07 - dense_101_loss: 5.3625e-07 - dense_102_loss: 5.9215e-07 - dense_103_loss: 5.8468e-07 - dense_104_loss: 7.0534e-07 - dense_105_loss: 7.9579e-07 - dense_106_loss: 1.0163e-06 - val_loss: 5.9934e-06 - val_dense_98_loss: 4.4950e-07 - val_dense_99_loss: 4.7145e-07 - val_dense_100_loss: 4.4066e-07 - val_dense_101_loss: 5.5883e-07 - val_dense_102_loss: 4.8719e-07 - val_dense_103_loss: 7.0430e-07 - val_dense_104_loss: 7.9290e-07 - val_dense_105_loss: 8.8162e-07 - val_dense_106_loss: 1.2070e-06
Epoch 269/500
2/2 [==============================] - 1s 282ms/step - loss: 5.7613e-06 - dense_98_loss: 4.1112e-07 - dense_99_loss: 5.7474e-07 - dense_100_loss: 5.4857e-07 - dense_101_loss: 5.3533e-07 - dense_102_loss: 5.9184e-07 - dense_103_loss: 5.8417e-07 - dense_104_loss: 7.0507e-07 - dense_105_loss: 7.9535e-07 - dense_106_loss: 1.0151e-06 - val_

2/2 [==============================] - 1s 290ms/step - loss: 5.7065e-06 - dense_98_loss: 4.1896e-07 - dense_99_loss: 5.7628e-07 - dense_100_loss: 5.4469e-07 - dense_101_loss: 5.3032e-07 - dense_102_loss: 5.8735e-07 - dense_103_loss: 5.7539e-07 - dense_104_loss: 6.9400e-07 - dense_105_loss: 7.7913e-07 - dense_106_loss: 1.0004e-06 - val_loss: 5.7816e-06 - val_dense_98_loss: 4.2295e-07 - val_dense_99_loss: 4.4570e-07 - val_dense_100_loss: 4.1523e-07 - val_dense_101_loss: 5.3648e-07 - val_dense_102_loss: 4.6514e-07 - val_dense_103_loss: 6.7924e-07 - val_dense_104_loss: 7.7035e-07 - val_dense_105_loss: 8.6056e-07 - val_dense_106_loss: 1.1859e-06
Epoch 293/500
2/2 [==============================] - 1s 220ms/step - loss: 5.5291e-06 - dense_98_loss: 3.9165e-07 - dense_99_loss: 5.5096e-07 - dense_100_loss: 5.2152e-07 - dense_101_loss: 5.1279e-07 - dense_102_loss: 5.6885e-07 - dense_103_loss: 5.5620e-07 - dense_104_loss: 6.7866e-07 - dense_105_loss: 7.6225e-07 - dense_106_loss: 9.8623e-07 - val_

2/2 [==============================] - 1s 215ms/step - loss: 6.4485e-06 - dense_98_loss: 5.4513e-07 - dense_99_loss: 6.7838e-07 - dense_100_loss: 6.4187e-07 - dense_101_loss: 6.2030e-07 - dense_102_loss: 6.7441e-07 - dense_103_loss: 6.5257e-07 - dense_104_loss: 7.5076e-07 - dense_105_loss: 8.3006e-07 - dense_106_loss: 1.0550e-06 - val_loss: 5.9010e-06 - val_dense_98_loss: 4.4451e-07 - val_dense_99_loss: 4.6287e-07 - val_dense_100_loss: 4.3059e-07 - val_dense_101_loss: 5.4855e-07 - val_dense_102_loss: 4.7888e-07 - val_dense_103_loss: 6.8998e-07 - val_dense_104_loss: 7.8025e-07 - val_dense_105_loss: 8.7211e-07 - val_dense_106_loss: 1.1933e-06
Epoch 317/500
2/2 [==============================] - 1s 300ms/step - loss: 5.5225e-06 - dense_98_loss: 4.1010e-07 - dense_99_loss: 5.5745e-07 - dense_100_loss: 5.2148e-07 - dense_101_loss: 5.1740e-07 - dense_102_loss: 5.7225e-07 - dense_103_loss: 5.5226e-07 - dense_104_loss: 6.7218e-07 - dense_105_loss: 7.4640e-07 - dense_106_loss: 9.7293e-07 - val_

Epoch 18/500
2/2 [==============================] - 0s 40ms/step - loss: 5.6715e-06 - val_loss: 5.0689e-06
Epoch 19/500
2/2 [==============================] - 0s 31ms/step - loss: 4.7737e-06 - val_loss: 4.6337e-06
Epoch 20/500
2/2 [==============================] - 0s 31ms/step - loss: 4.3489e-06 - val_loss: 4.3758e-06
Epoch 21/500
2/2 [==============================] - 0s 28ms/step - loss: 4.0985e-06 - val_loss: 4.1698e-06
Epoch 22/500
2/2 [==============================] - 0s 28ms/step - loss: 3.8915e-06 - val_loss: 3.8967e-06
Epoch 23/500
2/2 [==============================] - 0s 29ms/step - loss: 3.6174e-06 - val_loss: 3.5605e-06
Epoch 24/500
2/2 [==============================] - 0s 29ms/step - loss: 3.2859e-06 - val_loss: 3.2441e-06
Epoch 25/500
2/2 [==============================] - 0s 31ms/step - loss: 2.9783e-06 - val_loss: 2.9966e-06
Epoch 26/500
2/2 [==============================] - 0s 29ms/step - loss: 2.7406e-06 - val_loss: 2.8438e-06
Epoch 27/500
2/2 [===================

In [18]:
### Evaluate the Models for WP definition
def evaluate_models_train(histstruct):
    
    for histgroup in histnames:
        print('evaluating model for '+histgroup[0])
        print(histstruct.evaluate_classifier(histgroup)[0].shape)
    
    # get mse for training set
    if 'training' in histstruct.masks.keys(): masknames = ['dcson','highstat', 'training']
    else: masknames = ['dcson','highstat']
    mse_train = histstruct.get_scores_array( masknames=masknames )
    print('Found mse array for training set of following shape: {}'.format(mse_train.shape))
    
    # get mse for good set
    if 'good' in histstruct.masks.keys():
        mse_good = []
        for histname in histstruct.histnames:
            mse_good.append(histstruct.get_scores( histname=histname, masknames=['dcson','highstat','good'] ))
    else:
        mse_good = []
        for histname in histstruct.histnames:
            hists_good = hu.averagehists( histstruct.get_histograms( histname=histname, masknames=['dcson','highstat']), 1000 )
            thismse = histstruct.classifiers[histname].evaluate( hists_good )
            mse_good.append( thismse )
            print(run)
    mse_good = np.array(mse_good)
    mse_good = np.transpose(mse_good)
    print('Found mse array for good set of following shape: {}'.format(mse_good.shape))
    
    # get mse for bad sets
    mse_bad = []
    for i in range(nbadruns):
        mse_bad.append( histstruct.get_scores_array( masknames=['dcson','highstat','bad{}'.format(i)] ) )
        print('Found mse array for bad set of following shape: {}'.format(mse_bad[i].shape))
        
    return [mse_train, mse_good, mse_bad]

In [19]:
(mse_train, mse_good_eval, mse_bad_eval) = evaluate_models_train(histstruct)

evaluating model for chargeInner_PXLayer_1
(25676,)
evaluating model for charge_PXDisk_+1
(25676,)
evaluating model for charge_PXDisk_-1
(25676,)
evaluating model for Summary_ClusterStoNCorr__OnTrack__TIB__layer__1
(25676,)
evaluating model for Summary_ClusterStoNCorr__OnTrack__TOB__layer__1
(25676,)
evaluating model for Summary_ClusterStoNCorr__OnTrack__TID__PLUS__wheel__1
(25676,)
evaluating model for Summary_ClusterStoNCorr__OnTrack__TID__MINUS__wheel__1
(25676,)
evaluating model for Summary_ClusterStoNCorr__OnTrack__TEC__PLUS__wheel__1
(25676,)
evaluating model for Summary_ClusterStoNCorr__OnTrack__TEC__MINUS__wheel__1
(25676,)
evaluating model for NumberOfRecHitsPerTrack_lumiFlag_GenTk
(25676,)
Found mse array for training set of following shape: (19316, 45)
Found mse array for good set of following shape: (3415, 45)
Found mse array for bad set of following shape: (643, 45)
Found mse array for bad set of following shape: (105, 45)
Found mse array for bad set of following shape: (2

In [20]:
### Plots and Distribution Analysis
def fit_mse_distribution(histstruct, mse_train):
    dimslist = []
    fitfunclist = []
    
    
    nhisttypes = len(histstruct.histnames)
    for i in range(0,nhisttypes-1):
        for j in range(i+1,nhisttypes):
            dimslist.append((i, j))
    
    plt.close('all')
    (npoints,ndims) = mse_train.shape
    
    
    # settings for GaussianKdeFitter
    scott_bw = npoints**(-1./(ndims+4))
    bw_method = 20*scott_bw
    # settings for HyperRectangleFitter
    quantiles = ([0.00062,0.0006,0.00015,0.00015,
                 0.0003,0.0003,0.00053,0.00065])
    
    
    #for dims in dimslist:
    #    thismse = mse_train[:,dims]
    #    if training_mode=='global': 
    #        fitfunc = SeminormalFitter.SeminormalFitter(thismse)
    #        #fitfunc = HyperRectangleFitter.HyperRectangleFitter(thismse, 
    #        #                                                    [quantiles[dims[0]],quantiles[dims[1]]],
    #        #                                                    'up')
    #    else: fitfunc = GaussianKdeFitter.GaussianKdeFitter(thismse,bw_method=bw_method)
    #    #pu.plot_fit_2d(thismse, fitfunc=fitfunc, logprob=True, clipprob=True,
    #    #                onlycontour=False, xlims=30, ylims=30, 
    #    #                onlypositive=True, transparency=0.5,
    #    #                xaxtitle=histstruct.histnames[dims[0]], 
    #    #                yaxtitle=histstruct.histnames[dims[1]],
    #    #                title='density fit of lumisection MSE')
    #    ##plt.close('all') # release plot memory
    #    fitfunclist.append(fitfunc)
    # 
    #    
    if training_mode=='global': 
        fitfunc = SeminormalFitter.SeminormalFitter(mse_train)
        #fitfunc = HyperRectangleFitter.HyperRectangleFitter(mse_train, quantiles, 'up')
    else: 
        fitfunc = GaussianKdeFitter.GaussianKdeFitter()
        fitfunc.fit(mse_train,bw_method=bw_method)
    
    return fitfunc

In [21]:
fitfunc = fit_mse_distribution(histstruct, mse_train)

In [22]:
### Prepare MSEs for Working Point Definition
def mse_analysis(histstruct, mse_good_eval, mse_bad_eval, fitfunc):
    
    # Get the minimum log probability of histograms in good set
    print('--- good lumesections ---')
    logprob_good = np.log(fitfunc.pdf(mse_good_eval))
    print('length of log prob array: '+str(len(logprob_good)))
    print('minimum of log prob: '+str(np.min(logprob_good)))
    #print(sorted(logprob_good))
    
    print('--- bad lumisections ---')
    logprob_bad_parts = [np.log(fitfunc.pdf(mse_bad_eval[j])) for j in range(len(mse_bad_eval))]
    print(mse_bad_eval)
    #for lp in logprob_bad_parts: print(str(sorted(lp))+'\n\n')
    logprob_bad = np.concatenate(tuple(logprob_bad_parts))
    
    print('length of log prob array: '+str(len(logprob_bad)))
    print('maximum of log prob: '+str(np.max(logprob_bad)))
    #print(sorted(logprob_good))
    #print(sorted(logprob_bad))
    #print(logprob_bad)
    
    sep = np.min(logprob_good) - np.max(logprob_bad)
    print('Separability: ' + str(sep))
    
    return [logprob_good, logprob_bad]

In [23]:
(logprob_good, logprob_bad) = mse_analysis(histstruct, mse_good_eval, mse_bad_eval, fitfunc)

--- good lumesections ---
length of log prob array: 3415
minimum of log prob: 294.1874528736691
--- bad lumisections ---


/tmp/ipykernel_74415/3751926875.py:12: RuntimeWarning: divide by zero encountered in log
  logprob_bad_parts = [np.log(fitfunc.pdf(mse_bad_eval[j])) for j in range(len(mse_bad_eval))]


[array([[1.87150375e-05, 1.11432664e-05, 3.63185596e-06, ...,
        6.94788812e-05, 5.37784093e-05, 2.56079029e-05],
       [1.85777113e-05, 9.67499030e-06, 4.03235023e-06, ...,
        7.01211238e-05, 5.23477642e-05, 2.27661667e-05],
       [1.57784864e-05, 9.77928934e-06, 3.23648162e-06, ...,
        6.91592607e-05, 5.02742768e-05, 2.61399160e-05],
       ...,
       [5.78770465e-06, 2.30371919e-06, 1.90503082e-06, ...,
        3.36727923e-05, 2.33645025e-05, 2.41684304e-05],
       [7.37423546e-06, 2.89228078e-06, 2.14812124e-06, ...,
        3.25591991e-05, 2.54031555e-05, 2.68380744e-05],
       [7.73413812e-06, 2.84607878e-06, 2.10682689e-06, ...,
        3.30980979e-05, 2.78499002e-05, 2.80782097e-05]]), array([[2.32171746e-06, 1.26278404e-06, 2.01734513e-06, ...,
        1.40529741e-06, 2.43202923e-06, 1.92234758e-05],
       [1.28817649e-06, 2.00748118e-06, 3.09456966e-06, ...,
        2.35633798e-06, 4.23623727e-06, 2.54109174e-05],
       [1.90437295e-06, 1.11476380e-06, 1

In [24]:
def evaluate_autoencoders_combined(logprob_good, logprob_bad, fmBiasFactor, wpBiasFactor):
    labels_good = np.zeros(len(logprob_good)) # background: label = 0
    labels_bad = np.ones(len(logprob_bad)) # signal: label = 1
    
    badMin = min(np.where(logprob_bad != -np.inf, logprob_bad, -1))
    goodMax = max(np.where(logprob_good != np.inf, logprob_good, 10001))
    
    logprob_good = np.where(logprob_good != np.inf, logprob_good, goodMax)
    logprob_bad = np.where(logprob_bad != -np.inf, logprob_bad, badMin)
    
    # These only take effect if a histogram is grossly misclassified
    logprob_good[logprob_good == -np.inf] = badMin
    logprob_bad[logprob_bad == np.inf] = goodMax
    
    labels = np.loadtxt('Misclass.csv', delimiter = ",")
    scores = np.concatenate(tuple([-logprob_good,-logprob_bad]))
    scores = aeu.clip_scores( scores )
    
    avSep = np.mean(logprob_good) - np.mean(logprob_bad)
    
    print('Average Separation: ' + str(avSep))
    
    pu.plot_score_dist(scores, labels, siglabel='anomalous', sigcolor='r', 
                       bcklabel='good', bckcolor='g', 
                       nbins=200, normalize=True,
                       xaxtitle='negative logarithmic probability',
                       yaxtitle='number of lumisections (normalized)')
      
    # Plot ROC curve for analysis
    auc = aeu.get_roc(scores, labels, mode='geom', doprint=False)
    
    # Setting a threshold, below this working point defines anomalous data
    # Average is biased towards better recall per user specifications
    logprob_threshold = (1/(wpBiasFactor + 1)) * (wpBiasFactor*np.mean(logprob_good) + np.mean(logprob_bad))
    # Or set manual
    # logprob_threshold = 100
    (_, _, _, tp, fp, tn, fn) = aeu.get_confusion_matrix(scores,labels,-logprob_threshold)
    print('Selected logprob threshold of ' + str(logprob_threshold))
    
    # Get metrics for analysis
    accuracy = (tp + tn) / (tp + fp + tn + fn)
    precision = tp / (tp + fp)
    recall = tp / (tp + fn)
    f_measure = (1 + fmBiasFactor * fmBiasFactor) * ((precision * recall) / ((fmBiasFactor * fmBiasFactor * precision) + recall)) 
    
    print('Accuracy: ' + str(accuracy))
    print('Precision: ' + str(precision))
    print('Recall: ' + str(recall))
    print('F-Measure: ' + str(f_measure))
    
    return logprob_threshold

In [25]:
logprob_threshold = evaluate_autoencoders_combined(logprob_good, logprob_bad, fmBiasFactor, wpBiasFactor)

Average Separation: 38.42598010208593


IndexError: index 4611 is out of bounds for axis 0 with size 4611